In [3]:
import pandas as pd
from prophet import Prophet
import datetime as dt
import numpy as np
%matplotlib inline

In [4]:
df = pd.read_excel("./data/CCS Map Data Jan2023.xlsx").set_index("Country Location")
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 417 entries, United States to Canada
Data columns (total 38 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   Project ID                              417 non-null    int64         
 1   Project Name                            417 non-null    object        
 2   Company                                 416 non-null    object        
 3   Plant Name                              349 non-null    object        
 4   Storage and/or Capture                  404 non-null    object        
 5   Overall Status                          407 non-null    object        
 6   Plant Status                            283 non-null    object        
 7   Project Phase                           346 non-null    object        
 8   State Location                          356 non-null    object        
 9   Specific Site Location                  399 

In [5]:
df.columns

Index(['Project ID', 'Project Name', 'Company', 'Plant Name',
       'Storage and/or Capture', 'Overall Status', 'Plant Status',
       'Project Phase', 'State Location', 'Specific Site Location',
       'Size or Capture Amount/unit base power', 'Power or Capture Unit',
       'Power or Capture Unit Base Code', 'Combustion / Separation',
       'Capture Technology', 'Project Summary', 'Project Date', 'Cost',
       'Currency Name', 'Project Information Webpage', 'Latitude', 'Longitude',
       'Continent Code', 'Continent Name', 'Country Code', 'District Code',
       'DOE Support', 'Exact Checkbox', 'Exact', 'Paper',
       'Regional Partnership', 'Other', 'Scope Code', 'Scope Name',
       'State Code', 'Status Code', 'Project Type',
       'Injection Unit Base Code'],
      dtype='object')

In [6]:
dfNew = df[[
    "Project Date",
    "Country Code",
    "Project Name",
    "Size or Capture Amount/unit base power",
    "Latitude",
    "Longitude"
]]
dfNew

,Project Date,Country Code,Project Name,Size or Capture Amount/unit base power,Latitude,Longitude
Country Location,,,,,,
United States,NaT,USA,Lost Cabin Gas Plant CCS Project,NaN,43.293204,-107.602148
Romania,NaT,Rom,Getica CCS Demonstration Project,4109.999863,44.669722,23.407778
United States,2019-12-19,USA,Project Tundra Carbon Capture System,455.000000,47.065715,-101.214290
United States,2020-09-01,USA,North Dakota CarbonSAFE Phase III: Site Charac...,NaN,47.094600,-101.309539
United States,2019-10-01,USA,Front End Engineering Design of Linde-BASF Adv...,1000.000000,30.533711,-88.555873
...,...,...,...,...,...,...
France,NaT,FR,Axe Seine/Normandy decarbonization project,NaN,49.464000,0.466000
Iceland,NaT,Ice,CCS with Elkem Iceland,NaN,64.359210,-21.786830
United States,NaT,USA,Blue Planet's pilot plant,NaN,38.030740,-121.870060


In [7]:
dfNew["Size or Capture Amount/unit base power"] = dfNew["Size or Capture Amount/unit base power"].fillna(0)
dfNew

C:\Users\Lardex\AppData\Local\Temp\ipykernel_11212\2333859212.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfNew["Size or Capture Amount/unit base power"] = dfNew["Size or Capture Amount/unit base power"].fillna(0)


,Project Date,Country Code,Project Name,Size or Capture Amount/unit base power,Latitude,Longitude
Country Location,,,,,,
United States,NaT,USA,Lost Cabin Gas Plant CCS Project,0.000000,43.293204,-107.602148
Romania,NaT,Rom,Getica CCS Demonstration Project,4109.999863,44.669722,23.407778
United States,2019-12-19,USA,Project Tundra Carbon Capture System,455.000000,47.065715,-101.214290
United States,2020-09-01,USA,North Dakota CarbonSAFE Phase III: Site Charac...,0.000000,47.094600,-101.309539
United States,2019-10-01,USA,Front End Engineering Design of Linde-BASF Adv...,1000.000000,30.533711,-88.555873
...,...,...,...,...,...,...
France,NaT,FR,Axe Seine/Normandy decarbonization project,0.000000,49.464000,0.466000
Iceland,NaT,Ice,CCS with Elkem Iceland,0.000000,64.359210,-21.786830
United States,NaT,USA,Blue Planet's pilot plant,0.000000,38.030740,-121.870060


In [8]:
dfNew.to_csv("CCS Map Data.csv")